<a href="https://colab.research.google.com/github/FWWKCS/Machine-Learning/blob/main/05%EC%A3%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습

In [1]:
import numpy as np
### fit() 메서드는 아무 것도 수행하지 않고, predict()는 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측하는 단순한 분류기 생성
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass
    # predict 메서드는 단순히 Sex 피처가 1이면 0, 아니면 1로 예측
    def predict(self, X):
        pred = np.zeros( (X.shape[0],1) )
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1
        return pred

In [2]:
## 생성된 MyDummyClassifier를 이용해 타이타닉 생존자 예측 수행

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

# 1. 앞절의 예측 결과인 y_test, myprediction의 confusion matrix 출력


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할
try :
    titanic_df = pd.read_csv('./titanic_train.csv')
except :
    url = 'https://drive.google.com/uc?export=download&id=1N5C43TpIdPUe62wN26kxt1o14dQFffpq'
    titanic_df = pd.read_csv(url)


y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

# 1.1.MinMaxScaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
from sklearn.preprocessing import MinMaxScaler

features = ['Cabin','Sex','Embarked']
print(f'## before\n{X_titanic_df}')
le = MinMaxScaler()
le = le.fit(X_titanic_df)
for feature in features :
        X_titanic_df[feature] = le.transform(X_titanic_df)

# # 만약 모든 데이터 세트에 스케일링을 적용하려고하면..

# le = MinMaxScaler()
# le = le.fit(X_titanic_df)
# X_titanic_df = le.transform(X_titanic_df)

# # 데이터 정보(col)의 key값이 정수화된 nparray로 변하게되므로 커스텀 모델의 predict 함수에서 'Sex'키를 못찾는 오류가 발생
# # 그러면 키 매핑을 다시 해주면 작동
# titanic_col = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']
# X_titanic_df = pd.DataFrame(X_titanic_df, columns = titanic_col)
print(f'## after\n{X_titanic_df}')


X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용하여 학습/예측/평가 수행.
myclf = MyDummyClassifier()
myclf.fit(X_train ,y_train)

mypredictions = myclf.predict(X_test)

# 1.2. 앞절의 예측 결과인 y_test, myprediction의 confusion matrix 출력
       #1 line 실습내용
print(confusion_matrix(y_test, mypredictions))
print(f'Dummy Classifier의 정확도 : {accuracy_score(y_test, mypredictions):.4}')


## before
     Pclass  Sex        Age  SibSp  Parch     Fare  Cabin  Embarked
0         3    1  22.000000      1      0   7.2500      7         3
1         1    0  38.000000      1      0  71.2833      2         0
2         3    0  26.000000      0      0   7.9250      7         3
3         1    0  35.000000      1      0  53.1000      2         3
4         3    1  35.000000      0      0   8.0500      7         3
..      ...  ...        ...    ...    ...      ...    ...       ...
886       2    1  27.000000      0      0  13.0000      7         3
887       1    0  19.000000      0      0  30.0000      1         3
888       3    0  29.699118      1      2  23.4500      7         3
889       1    1  26.000000      0      0  30.0000      2         0
890       3    1  32.000000      0      0   7.7500      7         2

[891 rows x 8 columns]
## after
     Pclass  Sex       Age  SibSp     Parch      Fare  Cabin  Embarked
0       1.0  1.0  0.271174  0.125  0.000000  0.014151  0.875  1.000000

# 2. 로지스틱 회귀로 학습, 예측 및 평가수행, 예측 후 roc_auc_score 사용하여 ROC AUC 를 확인


In [10]:
def get_clf_eval(y_test=None, pred=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

try :
    diabetes_data = pd.read_csv('diabetes.csv')
except :
    url = 'https://drive.google.com/uc?export=download&id=1F2EXeb_p8HtMQY3f7LtzbJApJ3zzr7qA'
    diabetes_data = pd.read_csv(url)

print(diabetes_data['Outcome'].value_counts())
diabetes_data.head(3)

0    500
1    268
Name: Outcome, dtype: int64


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [16]:
X = diabetes_data.iloc[:, :-1]
y = diabetes_data.iloc[:, -1]

# standardscaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#  1.1. 학습데이터/테스트 데이터 분할 test_size=0.2, random_state=156, stratify=y 조건으로 X_train, X_test, y_train, y_test 로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, stratify = y, random_state = 156)

# 1.2.로지스틱 회귀로 학습, 예측 및 평가수행, 예측 후 get_clf_eval 사용하여 정확도 확인
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test) # 테스트케이스 문제 제공
get_clf_eval(y_test, pred) # 정답 데이터와 예측 데이터 비교

# 1.3 임계값 변화에 따른 예측 성능 확인
from sklearn.preprocessing import Binarizer
thresholds = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65]
pred_proba = lr_clf.predict_proba(X_test) # 예측 확률 생성

def get_eval_by_threshold(y_test , pred_proba_c1, thresholds):
    # thresholds 리스트 객체내의 값을 차례로 iteration하면서 Evaluation 수행.
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값:',custom_threshold)
        get_clf_eval(y_test , custom_predict)

get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

오차 행렬
[[90 10]
 [21 33]]
정확도: 0.7987, 정밀도: 0.7674, 재현율: 0.6111,    F1: 0.6804, AUC:0.7556
임곗값: 0.35
오차 행렬
[[76 24]
 [18 36]]
정확도: 0.7273, 정밀도: 0.6000, 재현율: 0.6667,    F1: 0.6316, AUC:0.7133
임곗값: 0.4
오차 행렬
[[81 19]
 [19 35]]
정확도: 0.7532, 정밀도: 0.6481, 재현율: 0.6481,    F1: 0.6481, AUC:0.7291
임곗값: 0.45
오차 행렬
[[85 15]
 [20 34]]
정확도: 0.7727, 정밀도: 0.6939, 재현율: 0.6296,    F1: 0.6602, AUC:0.7398
임곗값: 0.5
오차 행렬
[[90 10]
 [21 33]]
정확도: 0.7987, 정밀도: 0.7674, 재현율: 0.6111,    F1: 0.6804, AUC:0.7556
임곗값: 0.55
오차 행렬
[[91  9]
 [22 32]]
정확도: 0.7987, 정밀도: 0.7805, 재현율: 0.5926,    F1: 0.6737, AUC:0.7513
임곗값: 0.6
오차 행렬
[[92  8]
 [25 29]]
정확도: 0.7857, 정밀도: 0.7838, 재현율: 0.5370,    F1: 0.6374, AUC:0.7285
임곗값: 0.65
오차 행렬
[[93  7]
 [26 28]]
정확도: 0.7857, 정밀도: 0.8000, 재현율: 0.5185,    F1: 0.6292, AUC:0.7243
